# Mediapipe para detección de caras

In [ ]:
# Fuente https://github.com/googlesamples/mediapipe/blob/main/examples/face_landmarker/python/[MediaPipe_Python_Tasks]_Face_Landmarker.ipynb
# Utilidades visualización para el demostrador Face landmark detection

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [ ]:
# Carga de módulos
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Crea objeto FaceLandmarker 
# Obtener archivo .task y modificar la ruta 
base_options = python.BaseOptions(model_asset_path='C:/Users/otsed/Desktop/Docencia/VC/Modelos/face_landmarker.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

vid = cv2.VideoCapture(0)

while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()  
    if ret:
        # Adapta formato
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Detección
        detection_result = detector.detect(image)

        # Visualzia resultado
        annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

        # Visualziar blendshapes con gráfico de barras
        #plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])
        # Muestra fotograma
        cv2.imshow('Vid', annotated_image)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

# Mediapipe para detección de manos
Si bien es código tomado del respositorio, no va muy bien, detecta una única mano y no siempre, la demo online va mejor

In [ ]:
# Fuente https://github.com/googlesamples/mediapipe/blob/main/examples/hand_landmarker/python/hand_landmarker.ipynb
# Utilidades visualización para el demostrador Face landmark detection

import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


# Cámara
vid = cv2.VideoCapture(0)

# Crea objeto FaceLandmarker 
# Obtener archivo .task y modificar la ruta 
base_options = python.BaseOptions(model_asset_path='C:/Users/otsed/Desktop/Docencia/VC/Modelos/hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)
  
while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()
  
    if ret:          
        # Adapta formato
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # Detección
        detection_result = detector.detect(image)

        # Visualiza resultado
        annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

        # Muestra fotograma
        cv2.imshow('Vid', annotated_image)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Ejemplo alternativo, usando cvzone para la detección de manos con Mediapipe

In [ ]:
from cvzone.HandTrackingModule import HandDetector
import cv2

# Cámara
vid = cv2.VideoCapture(0)

# Initializa detector
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, minTrackCon=0.5)

while(True):      
    # Fotograma a fotograma
    ret, frame = vid.read()

    # Búsqueda de manos
    # 'draw' a True indica si se dibujan sobre la imagen 
    # 'flipType' a True para tratar la imagen reflejada
    hands, frame = detector.findHands(frame, draw=True, flipType=True)
    
    # Si hay manos detectadas
    if hands:
        # datos primera mano
        hand1 = hands[0]  
        lmList1 = hand1["lmList"]  # 21 landmarks
        bbox1 = hand1["bbox"]  # Contenedor (x,y,w,h)
        center1 = hand1['center']  # Centro
        handType1 = hand1["type"]  # identifica si es la mano derecha o izquierda

        # Contabiliza dedos extendidos de la mano
        fingers1 = detector.fingersUp(hand1)
        print(f'H1 = {fingers1.count(1)}', end=" ")  

        # Calcula distancia entre dos elementos concretos de la mano, dibujando segmento entre ellos
        length, info, frame = detector.findDistance(lmList1[8][0:2], lmList1[12][0:2], frame, color=(255, 0, 0),
                                                  scale=10)

        # Segunda mano presente?
        if len(hands) == 2:
            # Datos segunda mano
            hand2 = hands[1]
            lmList2 = hand2["lmList"]
            bbox2 = hand2["bbox"]
            center2 = hand2['center']
            handType2 = hand2["type"]

            # Contabiliza dedos extendidos de la mano
            fingers2 = detector.fingersUp(hand2)
            print(f'H2 = {fingers2.count(1)}', end=" ")

            # Calcula distancia entre dos elementos concretos de la mano, dibujando segmento entre ellos
            length, info, frame = detector.findDistance(lmList2[8][0:2], lmList2[12][0:2], frame, color=(255, 0, 0),
                                                      scale=10)

        print("\n")  

    # Display the image in a window
    cv2.imshow("Image", frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

# Mini ejemplo homografía enter dos imágenes
1) Marcar los mismos cuatro puntos y en el mismo orden en ambas imágenes
2) Moverse sobre la imagen de partida, ver resultado en el mapa del campo

In [ ]:

import numpy as np
import cv2
from skimage import transform

# Captura cuatro clics del ratón (parte uno)
def get_points(event, x, y, flags, param):
    puntos = param["points"]
    img = param["image"]

    if event == cv2.EVENT_LBUTTONDOWN:  # Botón izquierdo
        puntos.append((x, y))  
        
        # Dibujar en la imagen
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], img)

        # Cuarto puntos, condición de parada
        if len(puntos) == 4:
            cv2.destroyAllWindows()
            print("Coordenadas imagen:", puntos)


# Movimiento del puntero (parte 2)
def mouse_event(event, x, y, flags, param):
    homo = param["tform"]
    img = param["image"]

    if event == cv2.EVENT_MOUSEMOVE:  
        # Transformar posición del ratón a imagen destino
        punto_A = np.array([[x, y]])  # Puntero 
        punto_B = homo(punto_A)  # Transformar a imagen destino

        # Dibuja en la imagen destino
        imgtmp = img.copy()
        cv2.circle(imgtmp, (int(punto_B[0][0]), int(punto_B[0][1])), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], imgtmp)


############## 1. RECOPILA LOS CUATRO PUNTOS EN AMBAS IMÁGENES
#Inicializa listas depuntos
puntosA = []
puntosB = []

#Lee imágenes, crea copias de trabajo
vista = cv2.imread('EstadioDeGranCanaria.jpeg')
mapa = cv2.imread('EstadioCenital.jpg')
vistatmp = vista.copy()
mapatmp = mapa.copy()

#Vista del campo
cv2.imshow("Vista", vistatmp) 
params = {
    "points": puntosA,
    "image": vistatmp, 
    "wname": "Vista"
}
cv2.setMouseCallback("Vista", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)

#Vista ceintal
cv2.imshow("Mapa", mapa) 
params = {
    "points": puntosB,
    "image": mapatmp, 
    "wname": "Mapa"
}
cv2.setMouseCallback("Mapa", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)
cv2.destroyAllWindows()

#Transformación de los puntos https://scikit-image.org/docs/stable/auto_examples/transform/plot_transform_types.html
tform = transform.estimate_transform('projective', np.array(puntosA), np.array(puntosB))
tf_img = transform.warp(vista, tform.inverse, output_shape=(mapa.shape))

#Muestra imagen de entrada transformada
cv2.imshow("Homografía", tf_img) 

cv2.waitKey(-1)
cv2.destroyAllWindows()

###### 2. MUESTRA POSICIÓN DEL RATÓN EN EL CAMPO, EN LA IMAGEN DEL MAPA
# Mostrar las dos imágenes
cv2.imshow("Vista", vista)
cv2.imshow("Mapa", mapa)

params = {
    "tform": tform,
    "image": mapa, 
    "wname": "Mapa"
}

# Evento de movimiento del ratón
cv2.setMouseCallback("Vista", mouse_event, params)

cv2.waitKey(0)
cv2.destroyAllWindows()


# Detectando personas y esquinas en una imagen de campo de fútbol

In [ ]:
import cv2
import numpy as np

from ultralytics import YOLO  #Necesario para detectar con YOLO

#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height

#Carga modelo YOLO
model = YOLO('yolo11n.pt') #Contenedores
# Etiqueta de las clases o clase de interés
classNames = ["person"]

# Carga imagen
image = cv2.imread("CampoF1.jpeg")
height, width, _ = image.shape  
# Convierte a HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# Muestra entrada
cv2.imshow("Input", image)

# Rango de verde de interés en HSV
lower_green = np.array([35, 40, 40]) 
upper_green = np.array([85, 255, 255])

# Máscara de zona verde
mask = cv2.inRange(hsv, lower_green, upper_green)
# Muestra máscara
cv2.imshow("Máscara", mask)

# Contornos en la máscara
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Imagen vacía
final_mask = np.zeros_like(mask)

# Filtrar contornos con un área mayor a 500 píxeles
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 500:
        # Rellenar los contornos seleccionados
        cv2.drawContours(final_mask, [contour], -1, 255, thickness=cv2.FILLED)

#Muestra máscara resultado
cv2.imshow("MáscaraF", final_mask)

# Aplicar máscara sobre la imagen original
masked_image = cv2.bitwise_and(image, image, mask=final_mask)

#Aplica YOLO
results = model(masked_image) 
# Para cada detección la muestra
for r in results:
    boxes = r.boxes

    for box in boxes:
        # Contenedor
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(masked_image, (x1,y1), (x2,y2), (255, 255, 255), 2)

cv2.imshow("CampoF", masked_image)


# Convierte a grises y detecta de bordes
gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150)
#Muestra bordes
cv2.imshow("edges", edges)

# Detecta líneas con la transformada de Hough
min_line_length = gray.shape[0] // 3  # Longitud mínima de la linea
lineas_campo = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=30, minLineLength=min_line_length, maxLineGap=10)

# Filtrar las nlineas líneas más próximas a la parte inferior de la imagen
nlineas = 10
if lineas_campo is not None:
    lineas_campo = sorted(lineas_campo, key=lambda l: min(l[0][1], l[0][3]), reverse=True)
    lineas_campo = lineas_campo[:10]

# Inicializa grupos de líneas y umbral de orientación para separar
group_1 = []  
group_2 = []  
angle_threshold = 10

# Agrupa líneas por orientación
if lineas_campo is not None:
    for line in lineas_campo:
        x1, y1, x2, y2 = line[0]
        
        # Evita división por cero en líneas verticales
        if x2 - x1 == 0:
            angle = 90  
        else:
            # Ángulo en grados
            angle = np.degrees(np.arctan((y2 - y1) / (x2 - x1)))
            angle = abs(angle) 
        
        # Asignar grupo
        if abs(angle) < angle_threshold:  # Cercano a horizontal
            group_1.append(line)
        else:  # Resto
            group_2.append(line)

# Dibuja cada grupo con un color
if group_1:
    for line in group_1:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para grupo 1

if group_2:
    for line in group_2:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Verde para grupo 2

#INTERSECCIONES
# Obtener las intersecciones entre líneas de grupo diferente
image_height, image_width = image.shape[:2]
intersections = []

for i in range(len(group_1)):
    for j in range(len(group_2)):
        line1 = group_1[i][0]
        line2 = group_2[j][0]

        intersection = line_intersection(line1, line2)
        # Chequea que esté en la imagen
        if intersection and is_within_image(intersection, image_width, image_height):
            intersections.append(intersection)

# Dibuja  intersecciones en la imagen
for point in intersections:
    x, y = point
    cv2.circle(image, (int(x), int(y)), 5, (255, 0, 255), -1)

# Muestra imagen con líneas e intersecciones
cv2.imshow("Líneas y esquinas detectadas", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
